In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import joblib
import numpy as np
import os
from scipy.sparse import save_npz, vstack, hstack

try:
    stopwords.words("english")
except LookupError:
    print("NLTK stopwords indiriliyor...")
    nltk.download("stopwords")

INPUT_PARQUET_PATH = "..\\data\\raw\\labeled_reviews.parquet"
OUTPUT_DIR = "..\\data\\processed\\"

TEXT_ONLY_DIR = os.path.join(OUTPUT_DIR, "text_data")
HYBRID_DIR = os.path.join(OUTPUT_DIR, "hybrid")
os.makedirs(TEXT_ONLY_DIR, exist_ok=True)
os.makedirs(HYBRID_DIR, exist_ok=True)
INPUT_BASE_PATH = 'C:\\Users\\Aybars\\.cache\\kagglehub\\datasets\\naveedhn\\amazon-product-review-spam-and-non-spam\\versions\\1'
BASE_DIR = "C:\\work environment\\Projects\\amazon-spam-review\\"
OUTPUT_DIR = os.path.join(BASE_DIR, "data\\processed\\")
HYBRID_DIR = os.path.join(OUTPUT_DIR, "hybrid")

VECTORIZER_PATH = os.path.join(OUTPUT_DIR, "tfidf_vectorizer.joblib")

stop_words = set(stopwords.words("english"))
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def create_behavioral_features(df, reviewer_counts_train):
    df_out = pd.DataFrame(index=df.index)
    df_out['review_count'] = df['reviewerID'].map(reviewer_counts_train).fillna(0)
    return df_out.values

print("Kurulum tamamlandı.")

Kurulum tamamlandı.


In [2]:
import pandas as pd
import os
import shutil
import glob
from scipy.sparse import hstack, save_npz
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re

print("🚀 Tüm kategoriler için hibrit veri setini sıfırdan oluşturma işlemi başlıyor.")
print(f"Kaynak: Ham JSON verileri okunacak ({INPUT_BASE_PATH})")

# Işlenecek tüm kategori klasörlerinin tam yollarını bul
category_paths = glob.glob(os.path.join(INPUT_BASE_PATH, '*/'))

if not category_paths:
    print(f"🚨 HATA: '{INPUT_BASE_PATH}' altında hiçbir kategori klasörü bulunamadı. Lütfen yolu kontrol edin.")
else:
    all_categories = [os.path.basename(os.path.normpath(p)) for p in category_paths if os.path.isdir(p)]
    print(f"✅ Toplam {len(all_categories)} kategori bulundu: {all_categories}")

# --- 2. Ana Kategori Döngüsü ---
for category_name in all_categories:
    print(f"\n{'='*60}\n--- İşleniyor: {category_name} ---\n{'='*60}")
    
    category_output_dir = os.path.join(HYBRID_DIR, category_name)

    # Dosyaların zaten var olup olmadığını kontrol et, varsa bu kategoriyi atla.
    final_files_check = [os.path.join(category_output_dir, f) for f in ["X_train.npz", "X_test.npz", "y_train.npy", "y_test.npy"]]
    if all(os.path.exists(p) for p in final_files_check):
        print(f"✅ '{category_name}' için dosyalar zaten mevcut, bu kategori atlanıyor.")
        continue

    try:
        # <<< DEĞİŞİKLİK BURADA >>>
        # Klasörü silmek yerine, sadece var olduğundan emin oluyoruz.
        # exist_ok=True parametresi, klasör zaten varsa hata vermesini engeller.
        os.makedirs(category_output_dir, exist_ok=True)
        # <<< DEĞİŞİKLİĞİN SONU >>>

        # Doğrudan bulunan kategori adını kullanarak JSON dosyasını ara
        json_files = glob.glob(os.path.join(INPUT_BASE_PATH, category_name, "*.json"))
        if not json_files:
            print(f"🚨 UYARI: '{category_name}' içinde JSON dosyası bulunamadı. Atlanıyor.")
            continue
        
        file_path = json_files[0]
        print(f"💾 '{file_path}' okunuyor...")
        
        labeled_chunks = []
        chunk_iterator = pd.read_json(file_path, lines=True, chunksize=500000)
        for chunk in chunk_iterator:
            if "class" in chunk.columns:
                labeled_chunks.append(chunk.dropna(subset=['class']))
        
        if not labeled_chunks:
            print(f"🚨 UYARI: '{category_name}' içinde etiketli veri bulunamadı. Atlanıyor.")
            continue

        category_df = pd.concat(labeled_chunks, ignore_index=True)
        del labeled_chunks
        print(f"✅ '{category_name}' için {len(category_df)} etiketli yorum yüklendi.")

        print("🪓 Veri, eğitim ve test setlerine ayrılıyor...")
        train_df, test_df = train_test_split(
            category_df, test_size=0.25, random_state=42, stratify=category_df['class']
        )
        del category_df

        print("🏷️  Cevaplar (y) ayrılıyor...")
        y_train = train_df['class'].to_numpy()
        y_test = test_df['class'].to_numpy()
        train_df = train_df.drop(columns=['class'])
        test_df = test_df.drop(columns=['class'])

        print("🧹 Metin verileri temizleniyor...")
        train_df['cleaned_text'] = (train_df['summary'].astype(str) + ' ' + train_df['reviewText'].astype(str)).apply(clean_text)
        test_df['cleaned_text'] = (test_df['summary'].astype(str) + ' ' + train_df['reviewText'].astype(str)).apply(clean_text)

        print("📝 TF-IDF vektörleştirici SADECE eğitim verisiyle eğitiliyor...")
        tfidf_vectorizer = TfidfVectorizer(max_features=20000, ngram_range=(1, 2))
        X_train_tfidf = tfidf_vectorizer.fit_transform(train_df['cleaned_text'])
        X_test_tfidf = tfidf_vectorizer.transform(test_df['cleaned_text'])

        print("📊 Davranışsal özellikler oluşturuluyor...")
        local_reviewer_counts_train = train_df['reviewerID'].value_counts()
        X_train_behavioral = create_behavioral_features(train_df, local_reviewer_counts_train)
        X_test_behavioral = create_behavioral_features(test_df, local_reviewer_counts_train)

        print("➕ Özellikler birleştiriliyor ve kaydediliyor...")
        X_train = hstack([X_train_tfidf, X_train_behavioral]).tocsr()
        X_test = hstack([X_test_tfidf, X_test_behavioral]).tocsr()

        save_npz(os.path.join(category_output_dir, "X_train.npz"), X_train)
        save_npz(os.path.join(category_output_dir, "X_test.npz"), X_test)
        np.save(os.path.join(category_output_dir, "y_train.npy"), y_train)
        np.save(os.path.join(category_output_dir, "y_test.npy"), y_test)
        
        print(f"✅ '{category_name}' için sızıntısız veri seti başarıyla oluşturuldu.")

    except Exception as e:
        print(f"\n🚨 HATA: '{category_name}' işlenirken bir sorun oluştu: {e}")
        print("-> Sonraki kategoriye devam ediliyor...")
        continue

print(f"\n{'='*60}\n🎉 Tüm kategorilerin işlenmesi tamamlandı!\n{'='*60}")

🚀 Tüm kategoriler için hibrit veri setini sıfırdan oluşturma işlemi başlıyor.
Kaynak: Ham JSON verileri okunacak (C:\Users\Aybars\.cache\kagglehub\datasets\naveedhn\amazon-product-review-spam-and-non-spam\versions\1)
✅ Toplam 6 kategori bulundu: ['Cell_Phones_and_Accessories', 'Clothing_Shoes_and_Jewelry', 'Electronics', 'Home_and_Kitchen', 'Sports_and_Outdoors', 'Toys_and_Games']

--- İşleniyor: Cell_Phones_and_Accessories ---
✅ 'Cell_Phones_and_Accessories' için dosyalar zaten mevcut, bu kategori atlanıyor.

--- İşleniyor: Clothing_Shoes_and_Jewelry ---
✅ 'Clothing_Shoes_and_Jewelry' için dosyalar zaten mevcut, bu kategori atlanıyor.

--- İşleniyor: Electronics ---
💾 'C:\Users\Aybars\.cache\kagglehub\datasets\naveedhn\amazon-product-review-spam-and-non-spam\versions\1\Electronics\Electronics.json' okunuyor...
✅ 'Electronics' için 7574169 etiketli yorum yüklendi.
🪓 Veri, eğitim ve test setlerine ayrılıyor...
🏷️  Cevaplar (y) ayrılıyor...
🧹 Metin verileri temizleniyor...
📝 TF-IDF vektö

In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import joblib
import numpy as np
import os
from scipy.sparse import save_npz, vstack, hstack

try:
    stopwords.words("english")
except LookupError:
    print("NLTK stopwords indiriliyor...")
    nltk.download("stopwords")

INPUT_PARQUET_PATH = "..\\data\\raw\\labeled_reviews.parquet"
OUTPUT_DIR = "..\\data\\processed\\"

TEXT_ONLY_DIR = os.path.join(OUTPUT_DIR, "text_data")
HYBRID_DIR = os.path.join(OUTPUT_DIR, "hybrid")
os.makedirs(TEXT_ONLY_DIR, exist_ok=True)
os.makedirs(HYBRID_DIR, exist_ok=True)
INPUT_BASE_PATH = 'C:\\Users\\Aybars\\.cache\\kagglehub\\datasets\\naveedhn\\amazon-product-review-spam-and-non-spam\\versions\\1'
BASE_DIR = "C:\\work environment\\Projects\\amazon-spam-review\\"
OUTPUT_DIR = os.path.join(BASE_DIR, "data\\processed\\")
HYBRID_DIR = os.path.join(OUTPUT_DIR, "hybrid")

VECTORIZER_PATH = os.path.join(OUTPUT_DIR, "tfidf_vectorizer.joblib")

stop_words = set(stopwords.words("english"))
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def create_behavioral_features(df, reviewer_counts_train):
    df_out = pd.DataFrame(index=df.index)
    df_out['review_count'] = df['reviewerID'].map(reviewer_counts_train).fillna(0)
    return df_out.values

print("Kurulum tamamlandı.")

In [ ]:
print("Text-only veri seti oluşturuluyor...")

# Parquet dosyasından sadece gerekli kolonları oku
df = pd.read_parquet(INPUT_PARQUET_PATH, columns=['summary', 'reviewText', 'class'])

# Metin verilerini birleştir ve temizle
df['full_text'] = (df['summary'].astype(str) + ' ' + df['reviewText'].astype(str))
df['cleaned_text'] = df['full_text'].apply(clean_text)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    df['cleaned_text'],
    df['class'],
    test_size=0.25,
    random_state=42,
    stratify=df['class']
)

# TF-IDF vektörizasyonu
tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1, 2))
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

# Dosyaları kaydet
os.makedirs(TEXT_ONLY_DIR, exist_ok=True)
save_npz(os.path.join(TEXT_ONLY_DIR, "X_train.npz"), X_train)
save_npz(os.path.join(TEXT_ONLY_DIR, "X_test.npz"), X_test)
np.save(os.path.join(TEXT_ONLY_DIR, "y_train.npy"), y_train.values)
np.save(os.path.join(TEXT_ONLY_DIR, "y_test.npy"), y_test.values)

print(f"✅ Text-only veri seti oluşturuldu. Boyutlar: Train {X_train.shape}, Test {X_test.shape}")

# Hybrid veri seti işlemleri buradan devam eder...